In [1]:
! pip -V

pip 22.3.1 from /Users/makroguzov/Desktop/posos/.venv/lib/python3.10/site-packages/pip (python 3.10)


In [ ]:
! pip install keras
! pip install scikit-learn
! pip install pandas
! pip install nltk
! pip install --upgrade pip
! pip install tensorflow

In [227]:
import json
import pprint
import pandas as pd
import string
import nltk
import re
import keras
import numpy as np


nltk.download('stopwords')

ModuleNotFoundError: No module named 'tensorflow'

In [4]:
DATA_PATH = './movies'

In [5]:
M_PATH = f'{DATA_PATH}/movies.json' #
U_PATH = f'{DATA_PATH}/users'       # 

In [53]:
def import_movies(m_path):
    """Генератор фильмов. Возвращаю словарик с фильмами"""
    with open(m_path, 'r', encoding='utf-8') as f:
        data = f.read()
        for movie in json.loads(data).values():
            if movie['description']:
                yield {
                    'id': movie['id'],
                    'name': movie['name'],
                    'russian': movie['russian'],
                    'score': movie['score'],
                    'description': movie['description'],
                }

In [54]:
def import_genres(m_path):
    """Генератор собирающий все жанры по фильмам"""
    with open(m_path, 'r', encoding='utf-8') as f:
        data = f.read()
        for movie in json.loads(data).values():
            for genre in movie['genres']:
                genre['movie_id'] = movie['id']
                yield genre

In [55]:
MOVIES = pd.DataFrame(list(import_movies(M_PATH)))
MOVIES.head(3)

,id,name,russian,score,description
0,4884,Tales of the Abyss,Сказания Бездны,7.29,"Люк фон Фабр, избалованный отпрыск благородных..."
1,4896,Umineko no Naku Koro ni,Когда плачут чайки,7.09,"Действие происходит на острове Роккэнджима, пр..."
2,4898,Kuroshitsuji,Тёмный дворецкий,7.7,События происходят в Англии викторианской эпох...


In [56]:
GENRES = pd.DataFrame(list(import_genres(M_PATH)))
GENRES.head(3)

,id,name,russian,kind,movie_id
0,2,Adventure,Приключения,anime,4884
1,8,Drama,Драма,anime,4884
2,10,Fantasy,Фэнтези,anime,4884


In [137]:
from nltk.corpus import stopwords

RUSSIAN_STOP_WORDS = stopwords.words('russian')


def remove_stop_words(text):
    return ' '.join([word for word in text.split(' ') if word not in RUSSIAN_STOP_WORDS])

In [147]:
pattern = re.compile(r'[^А-яЁё]+')


def remove_others(text):
    return pattern.sub(' ', text).strip()

In [148]:
MOVIES['p_description'] = MOVIES['description']
# Преобразования текста
MOVIES['p_description'] = MOVIES['p_description'].apply(lambda text: remove_others(text))
MOVIES['p_description'] = MOVIES['p_description'].apply(lambda text: remove_stop_words(text))

In [168]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(analyzer='word').fit(list(MOVIES['p_description']))
tfidf

TfidfVectorizer()

In [219]:
X = tfidf_wm.transform(MOVIES['p_description']).toarray()
y = MOVIES['score']

In [202]:
MOVIES['p_description'].shape

(6836,)

In [214]:
X.shape

(6836, 63195)

In [215]:
X

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [217]:
np.sum(X, axis=1)

array([8.03959505, 9.25096161, 7.59446908, ..., 9.63429795, 6.54867295,
       9.05075462])

In [221]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    train_size=0.67, 
                                                    random_state=42)

In [222]:
from keras import layers


ModuleNotFoundError: No module named 'tensorflow'